## 노인 사고가 많은 곳은 어디?

In [4]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import os
import webbrowser
import folium
from folium import plugins
import json

geo_path = 'data\\02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

prot_df = pd.read_csv('data\\seoul_protection_facility.csv', index_col = 0)
accident_df =  pd.read_csv('data\\total.csv',header=None)

#filtering
try:
    accident_df = accident_df.drop(0 ,axis=0)
    accident_df = accident_df.drop(1 ,axis=0)
except:
    pass

# Edit index
accident_df = accident_df.set_index(7)

df_total = pd.concat([accident_df, prot_df],  axis = 1)[[6,'노인','어린이']]
df_total[6]=df_total[6].astype(int)

map_acc = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                 tiles='Stamen Toner')
map_acc.choropleth(geo_data = geo_str,
               data = df_total[6],
               columns = [df_total.index, df_total[6]],
               fill_color = 'PuRd', #puRd, YlGnBu
               key_on = 'feature.id')


C:\Users\Hwang\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

C:\Users\Hwang\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


## 사각지대

In [5]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import os
import webbrowser
import folium
from folium import plugins
import json

geo_path = 'data\\02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

prot_df = pd.read_csv('data\\seoul_protection_facility.csv', index_col = 0)
accident_df =  pd.read_csv('data\\total.csv',header=None)

#filtering
try:
    accident_df = accident_df.drop(0 ,axis=0)
    accident_df = accident_df.drop(1 ,axis=0)
except:
    pass

# Edit index
accident_df = accident_df.set_index(7)

df_total = pd.concat([accident_df, prot_df],  axis = 1)[[6,'노인','어린이']]
df_total[6]=df_total[6].astype(int)

df_ratio = df_total['노인'].astype(int).div(df_total[6]).replace([np.inf, -np.inf],0)
df_ratio = df_ratio.to_frame('ratio')
map_blind = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                 tiles='Stamen Toner')
map_blind.choropleth(geo_data = geo_str,
               data = df_ratio['ratio'],
               columns = [df_ratio.index, df_ratio['ratio']],
               fill_color = 'PuRd', #puRd, YlGnBu
               key_on = 'feature.id')

C:\Users\Hwang\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

C:\Users\Hwang\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [6]:
import pandas as pd
import numpy as np
old = pd.read_csv("data\\전국노인장애인보호구역표준데이터.csv",encoding = 'cp949',engine = 'python')
old_seoul = old[old['시도명'] == '서울특별시']
old_seoul.rename(columns = {'시군구명' : '지역구'}, inplace = True)
old_seoul.loc[old_seoul["지역구"] == "서울특별시 노원구","지역구"] = '노원구'
X_Y = old_seoul[['지역구','위도','경도']]

young = pd.read_csv('data\\전국어린이보호구역표준데이터.csv', encoding = 'cp949',engine='python')
young_seoul = young.loc[young['소재지도로명주소'].str.contains('서울특별시',na=False)]
young_seoul['자치구'] = young_seoul['소재지도로명주소'].str.split(" ").str[1]
young_seoul['자치구'] = young_seoul['자치구'].str.replace('덕릉로62길','도봉구')

young_seoul = young_seoul.rename({'지역구':'자치구'},axis='columns')
X_Y_young = young_seoul[['자치구','위도','경도']]


import folium 
from folium import plugins

m = folium.Map(location=[37.5502, 126.982], 
                           tiles='cartodbpositron', zoom_start=10) 

folium.TileLayer('openstreetmap').add_to(m) 

fg = folium.FeatureGroup(name='보호구역') 
m.add_child(fg)


g1 = plugins.FeatureGroupSubGroup(fg, '노인 보호구역') 
m.add_child(g1)
g2 = plugins.FeatureGroupSubGroup(fg, '어린이 보호구역')
m.add_child(g2)

for row in X_Y.values:
    folium.Circle(
      [row[1],row[2]],
      radius=200,
      color = 'red'
    ).add_to(g1)  

for row in X_Y_young.values:
    folium.Circle(
      [row[1],row[2]],
      radius=200,
    ).add_to(g2)    
    

folium.LayerControl(collapsed=False).add_to(m)

C:\Users\Hwang\Anaconda3\lib\site-packages\pandas\core\frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
C:\Users\Hwang\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Hwang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/ind

In [7]:
m